# Data preparation

Load data from CSV files and export to database\
CSV files downloaded from https://www.kaggle.com/martinellis/nhl-game-data/

In [1]:
import os
os.environ["JAVA_HOME"] = "/Library/Java/JavaVirtualMachines/jdk1.8.0_231.jdk/Contents/Home/"
os.environ["JRE_HOME"] = "/Library/Java/JavaVirtualMachines/jdk1.8.0_231.jdk/Contents/Home/"
import findspark
findspark.init("/usr/local/Cellar/apache-spark@2.3.2/2.3.2/libexec/")
import pyspark
sc = pyspark.SparkContext()
spark = pyspark.sql.SQLContext(sc)
spark.sql("SET @@global.time_zone='+00:00'")

DataFrame[key: string, value: string]

Individual game information

In [3]:
game_file_path = '../../data/game.csv'
df_raw = spark.read.csv(game_file_path, 
                        header=True, 
                        mode="DROPMALFORMED")
df_raw.printSchema()
df_raw.createOrReplaceTempView('game_raw')
spark.sql("""
SELECT * FROM game_raw LIMIT 10
""").show()

root
 |-- game_id: string (nullable = true)
 |-- season: string (nullable = true)
 |-- type: string (nullable = true)
 |-- date_time: string (nullable = true)
 |-- date_time_GMT: string (nullable = true)
 |-- away_team_id: string (nullable = true)
 |-- home_team_id: string (nullable = true)
 |-- away_goals: string (nullable = true)
 |-- home_goals: string (nullable = true)
 |-- outcome: string (nullable = true)
 |-- home_rink_side_start: string (nullable = true)
 |-- venue: string (nullable = true)
 |-- venue_link: string (nullable = true)
 |-- venue_time_zone_id: string (nullable = true)
 |-- venue_time_zone_offset: string (nullable = true)
 |-- venue_time_zone_tz: string (nullable = true)

+----------+--------+----+----------+--------------------+------------+------------+----------+----------+------------+--------------------+------------------+-------------------+-------------------+----------------------+------------------+
|   game_id|  season|type| date_time|       date_time_GMT

In [15]:
df = spark.sql("""
SELECT game_id, CAST(season AS int), type, CAST(date_time AS date), CAST(away_team_id AS int), CAST(home_team_id AS int), CAST(away_goals AS int), CAST(home_goals AS int), outcome, home_rink_side_start, venue, venue_time_zone_id, CAST(venue_time_zone_offset AS int)
FROM game_raw
""")
df.printSchema()

root
 |-- game_id: string (nullable = true)
 |-- season: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- date_time: date (nullable = true)
 |-- away_team_id: integer (nullable = true)
 |-- home_team_id: integer (nullable = true)
 |-- away_goals: integer (nullable = true)
 |-- home_goals: integer (nullable = true)
 |-- outcome: string (nullable = true)
 |-- home_rink_side_start: string (nullable = true)
 |-- venue: string (nullable = true)
 |-- venue_time_zone_id: string (nullable = true)
 |-- venue_time_zone_offset: integer (nullable = true)



In [14]:
df.write.format('jdbc').options(
    url='jdbc:mysql://localhost/50_in_07',
    driver='com.mysql.jdbc.Driver',
    dbtable='game',
    user='root').mode('overwrite').save()

Goalie information

In [17]:
goalie_file_path = '../../data/game_goalie_stats.csv'
df_raw = spark.read.csv(goalie_file_path, 
                        header=True, 
                        mode="DROPMALFORMED")
df_raw.printSchema()
df_raw.createOrReplaceTempView('goalie_raw')
spark.sql("""
SELECT * FROM goalie_raw LIMIT 10
""").show()

root
 |-- game_id: string (nullable = true)
 |-- player_id: string (nullable = true)
 |-- team_id: string (nullable = true)
 |-- timeOnIce: string (nullable = true)
 |-- assists: string (nullable = true)
 |-- goals: string (nullable = true)
 |-- pim: string (nullable = true)
 |-- shots: string (nullable = true)
 |-- saves: string (nullable = true)
 |-- powerPlaySaves: string (nullable = true)
 |-- shortHandedSaves: string (nullable = true)
 |-- evenSaves: string (nullable = true)
 |-- shortHandedShotsAgainst: string (nullable = true)
 |-- evenShotsAgainst: string (nullable = true)
 |-- powerPlayShotsAgainst: string (nullable = true)
 |-- decision: string (nullable = true)
 |-- savePercentage: string (nullable = true)
 |-- powerPlaySavePercentage: string (nullable = true)
 |-- evenStrengthSavePercentage: string (nullable = true)

+----------+---------+-------+---------+-------+-----+---+-----+-----+--------------+----------------+---------+-----------------------+----------------+------

In [18]:
df = spark.sql("""
SELECT game_id, CAST(player_id AS int), CAST(team_id AS int), CAST(timeOnIce AS int), CAST(assists AS int), CAST(goals AS int), CAST(pim AS int), CAST(shots AS int), CAST(saves AS int), CAST(powerPlaySaves AS int), CAST(shortHandedSaves AS int), CAST(evenSaves AS int), CAST(shortHandedShotsAgainst AS int), CAST(evenShotsAgainst AS int), CAST(powerPlayShotsAgainst AS int), decision, CAST(savePercentage AS float), CAST(powerPlaySavePercentage AS float), CAST(evenStrengthSavePercentage AS float)
FROM goalie_raw
""")
df.printSchema()

root
 |-- game_id: string (nullable = true)
 |-- player_id: integer (nullable = true)
 |-- team_id: integer (nullable = true)
 |-- timeOnIce: integer (nullable = true)
 |-- assists: integer (nullable = true)
 |-- goals: integer (nullable = true)
 |-- pim: integer (nullable = true)
 |-- shots: integer (nullable = true)
 |-- saves: integer (nullable = true)
 |-- powerPlaySaves: integer (nullable = true)
 |-- shortHandedSaves: integer (nullable = true)
 |-- evenSaves: integer (nullable = true)
 |-- shortHandedShotsAgainst: integer (nullable = true)
 |-- evenShotsAgainst: integer (nullable = true)
 |-- powerPlayShotsAgainst: integer (nullable = true)
 |-- decision: string (nullable = true)
 |-- savePercentage: float (nullable = true)
 |-- powerPlaySavePercentage: float (nullable = true)
 |-- evenStrengthSavePercentage: float (nullable = true)



In [19]:
df.write.format('jdbc').options(
    url='jdbc:mysql://localhost/50_in_07',
    driver='com.mysql.jdbc.Driver',
    dbtable='goalie',
    user='root').mode('overwrite').save()

Skater information

In [23]:
skater_file_path = '../../data/game_skater_stats.csv'
df_raw = spark.read.csv(skater_file_path, 
                        header=True, 
                        mode="DROPMALFORMED")
df_raw.printSchema()
df_raw.createOrReplaceTempView('skater_raw')
spark.sql("""
SELECT * FROM skater_raw LIMIT 10
""").show()

root
 |-- game_id: string (nullable = true)
 |-- player_id: string (nullable = true)
 |-- team_id: string (nullable = true)
 |-- timeOnIce: string (nullable = true)
 |-- assists: string (nullable = true)
 |-- goals: string (nullable = true)
 |-- shots: string (nullable = true)
 |-- hits: string (nullable = true)
 |-- powerPlayGoals: string (nullable = true)
 |-- powerPlayAssists: string (nullable = true)
 |-- penaltyMinutes: string (nullable = true)
 |-- faceOffWins: string (nullable = true)
 |-- faceoffTaken: string (nullable = true)
 |-- takeaways: string (nullable = true)
 |-- giveaways: string (nullable = true)
 |-- shortHandedGoals: string (nullable = true)
 |-- shortHandedAssists: string (nullable = true)
 |-- blocked: string (nullable = true)
 |-- plusMinus: string (nullable = true)
 |-- evenTimeOnIce: string (nullable = true)
 |-- shortHandedTimeOnIce: string (nullable = true)
 |-- powerPlayTimeOnIce: string (nullable = true)

+----------+---------+-------+---------+-------+---

In [26]:
df = spark.sql("""
SELECT game_id, CAST(player_id AS int), CAST(team_id AS int), CAST(timeOnIce AS int), CAST(assists AS int), CAST(goals AS int), CAST(shots AS int), CAST(hits AS int), CAST(powerPlayGoals AS int), CAST(powerPlayAssists AS int), CAST(penaltyMinutes AS int), CAST(faceOffWins AS int), CAST(faceoffTaken AS int) AS faceOffTaken, CAST(takeaways AS int), CAST(giveaways AS int), CAST(shortHandedGoals AS int), CAST(shortHandedAssists AS int), CAST(blocked AS int), CAST(plusMinus AS int), CAST(evenTimeOnIce AS int), CAST(shortHandedTimeOnIce AS int), CAST(powerPlayTimeOnIce AS int)
FROM skater_raw
""")
df.printSchema()

root
 |-- game_id: string (nullable = true)
 |-- player_id: integer (nullable = true)
 |-- team_id: integer (nullable = true)
 |-- timeOnIce: integer (nullable = true)
 |-- assists: integer (nullable = true)
 |-- goals: integer (nullable = true)
 |-- shots: integer (nullable = true)
 |-- hits: integer (nullable = true)
 |-- powerPlayGoals: integer (nullable = true)
 |-- powerPlayAssists: integer (nullable = true)
 |-- penaltyMinutes: integer (nullable = true)
 |-- faceOffWins: integer (nullable = true)
 |-- faceOffTaken: integer (nullable = true)
 |-- takeaways: integer (nullable = true)
 |-- giveaways: integer (nullable = true)
 |-- shortHandedGoals: integer (nullable = true)
 |-- shortHandedAssists: integer (nullable = true)
 |-- blocked: integer (nullable = true)
 |-- plusMinus: integer (nullable = true)
 |-- evenTimeOnIce: integer (nullable = true)
 |-- shortHandedTimeOnIce: integer (nullable = true)
 |-- powerPlayTimeOnIce: integer (nullable = true)



In [25]:
df.write.format('jdbc').options(
    url='jdbc:mysql://localhost/50_in_07',
    driver='com.mysql.jdbc.Driver',
    dbtable='skater',
    user='root').mode('overwrite').save()

Team information

In [29]:
teams_file_path = '../../data/game_teams_stats.csv'
df_raw = spark.read.csv(teams_file_path, 
                        header=True, 
                        mode="DROPMALFORMED")
df_raw.printSchema()
df_raw.createOrReplaceTempView('teams_raw')
spark.sql("""
SELECT * FROM teams_raw LIMIT 10
""").show()

root
 |-- game_id: string (nullable = true)
 |-- team_id: string (nullable = true)
 |-- HoA: string (nullable = true)
 |-- won: string (nullable = true)
 |-- settled_in: string (nullable = true)
 |-- head_coach: string (nullable = true)
 |-- goals: string (nullable = true)
 |-- shots: string (nullable = true)
 |-- hits: string (nullable = true)
 |-- pim: string (nullable = true)
 |-- powerPlayOpportunities: string (nullable = true)
 |-- powerPlayGoals: string (nullable = true)
 |-- faceOffWinPercentage: string (nullable = true)
 |-- giveaways: string (nullable = true)
 |-- takeaways: string (nullable = true)

+----------+-------+----+-----+----------+----------------+-----+-----+----+---+----------------------+--------------+--------------------+---------+---------+
|   game_id|team_id| HoA|  won|settled_in|      head_coach|goals|shots|hits|pim|powerPlayOpportunities|powerPlayGoals|faceOffWinPercentage|giveaways|takeaways|
+----------+-------+----+-----+----------+----------------+----

In [30]:
df = spark.sql("""
SELECT game_id, CAST(team_id AS int), HoA, won, settled_in, head_coach, CAST(goals AS int), CAST(shots AS int), CAST(hits AS int), CAST(pim AS int), CAST(powerPlayOpportunities AS int), CAST(powerPlayGoals AS int), CAST(faceOffWinPercentage AS float), CAST(giveaways AS int), CAST(takeaways AS int)
FROM teams_raw
""")
df.printSchema()

root
 |-- game_id: string (nullable = true)
 |-- team_id: integer (nullable = true)
 |-- HoA: string (nullable = true)
 |-- won: string (nullable = true)
 |-- settled_in: string (nullable = true)
 |-- head_coach: string (nullable = true)
 |-- goals: integer (nullable = true)
 |-- shots: integer (nullable = true)
 |-- hits: integer (nullable = true)
 |-- pim: integer (nullable = true)
 |-- powerPlayOpportunities: integer (nullable = true)
 |-- powerPlayGoals: integer (nullable = true)
 |-- faceOffWinPercentage: float (nullable = true)
 |-- giveaways: integer (nullable = true)
 |-- takeaways: integer (nullable = true)



In [31]:
df.write.format('jdbc').options(
    url='jdbc:mysql://localhost/50_in_07',
    driver='com.mysql.jdbc.Driver',
    dbtable='teams',
    user='root').mode('overwrite').save()

Individual player information

In [32]:
player_info_file_path = '../../data/player_info.csv'
df_raw = spark.read.csv(player_info_file_path, 
                        header=True, 
                        mode="DROPMALFORMED")
df_raw.printSchema()
df_raw.createOrReplaceTempView('player_info_raw')
spark.sql("""
SELECT * FROM player_info_raw LIMIT 10
""").show()

root
 |-- player_id: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- lastName: string (nullable = true)
 |-- nationality: string (nullable = true)
 |-- birthCity: string (nullable = true)
 |-- primaryPosition: string (nullable = true)
 |-- birthDate: string (nullable = true)
 |-- link: string (nullable = true)

+---------+---------+------------+-----------+-----------+---------------+----------+--------------------+
|player_id|firstName|    lastName|nationality|  birthCity|primaryPosition| birthDate|                link|
+---------+---------+------------+-----------+-----------+---------------+----------+--------------------+
|  8467412|   Alexei|Ponikarovsky|        UKR|       Kiev|             LW|1980-04-09|/api/v1/people/84...|
|  8468501|    Anton|  Volchenkov|        RUS|     Moscow|              D|1982-02-25|/api/v1/people/84...|
|  8459670|    Kimmo|     Timonen|        FIN|     Kuopio|              D|1975-03-18|/api/v1/people/84...|
|  8471233|   Travis| 

In [33]:
df = spark.sql("""
SELECT CAST(player_id AS int), firstName, lastName, nationality, birthCity, primaryPosition, CAST(birthDate AS date), link
FROM player_info_raw
""")
df.printSchema()

root
 |-- player_id: integer (nullable = true)
 |-- firstName: string (nullable = true)
 |-- lastName: string (nullable = true)
 |-- nationality: string (nullable = true)
 |-- birthCity: string (nullable = true)
 |-- primaryPosition: string (nullable = true)
 |-- birthDate: date (nullable = true)
 |-- link: string (nullable = true)



In [34]:
df.write.format('jdbc').options(
    url='jdbc:mysql://localhost/50_in_07',
    driver='com.mysql.jdbc.Driver',
    dbtable='player_info',
    user='root').mode('overwrite').save()

In [35]:
team_info_file_path = '../../data/team_info.csv'
df_raw = spark.read.csv(team_info_file_path, 
                        header=True, 
                        mode="DROPMALFORMED")
df_raw.printSchema()
df_raw.createOrReplaceTempView('team_info_raw')
spark.sql("""
SELECT * FROM team_info_raw LIMIT 10
""").show()

root
 |-- team_id: string (nullable = true)
 |-- franchiseId: string (nullable = true)
 |-- shortName: string (nullable = true)
 |-- teamName: string (nullable = true)
 |-- abbreviation: string (nullable = true)
 |-- link: string (nullable = true)

+-------+-----------+------------+---------+------------+----------------+
|team_id|franchiseId|   shortName| teamName|abbreviation|            link|
+-------+-----------+------------+---------+------------+----------------+
|      1|         23|  New Jersey|   Devils|         NJD| /api/v1/teams/1|
|      4|         16|Philadelphia|   Flyers|         PHI| /api/v1/teams/4|
|     26|         14| Los Angeles|    Kings|         LAK|/api/v1/teams/26|
|     14|         31|   Tampa Bay|Lightning|         TBL|/api/v1/teams/14|
|      6|          6|      Boston|   Bruins|         BOS| /api/v1/teams/6|
|      3|         10|  NY Rangers|  Rangers|         NYR| /api/v1/teams/3|
|      5|         17|  Pittsburgh| Penguins|         PIT| /api/v1/teams/5|
|

In [36]:
df = spark.sql("""
SELECT CAST(team_id AS int), CAST(franchiseID AS int), shortName, teamName, abbreviation, link
FROM team_info_raw
""")
df.printSchema()

root
 |-- team_id: integer (nullable = true)
 |-- franchiseID: integer (nullable = true)
 |-- shortName: string (nullable = true)
 |-- teamName: string (nullable = true)
 |-- abbreviation: string (nullable = true)
 |-- link: string (nullable = true)



In [37]:
df.write.format('jdbc').options(
    url='jdbc:mysql://localhost/50_in_07',
    driver='com.mysql.jdbc.Driver',
    dbtable='team_info',
    user='root').mode('overwrite').save()

Play information

In [38]:
plays_file_path = '../../data/game_plays.csv'
df_raw = spark.read.csv(plays_file_path, 
                        header=True, 
                        mode="DROPMALFORMED")
df_raw.printSchema()
df_raw.createOrReplaceTempView('plays_raw')
spark.sql("""
SELECT * FROM plays_raw LIMIT 10
""").show()

root
 |-- play_id: string (nullable = true)
 |-- game_id: string (nullable = true)
 |-- play_num: string (nullable = true)
 |-- team_id_for: string (nullable = true)
 |-- team_id_against: string (nullable = true)
 |-- event: string (nullable = true)
 |-- secondaryType: string (nullable = true)
 |-- x: string (nullable = true)
 |-- y: string (nullable = true)
 |-- period: string (nullable = true)
 |-- periodType: string (nullable = true)
 |-- periodTime: string (nullable = true)
 |-- periodTimeRemaining: string (nullable = true)
 |-- dateTime: string (nullable = true)
 |-- goals_away: string (nullable = true)
 |-- goals_home: string (nullable = true)
 |-- description: string (nullable = true)
 |-- st_x: string (nullable = true)
 |-- st_y: string (nullable = true)
 |-- rink_side: string (nullable = true)

+-------------+----------+--------+-----------+---------------+--------------+-------------+---+---+------+----------+----------+-------------------+-------------------+----------+-----

In [73]:
df = spark.sql("""
SELECT play_id, game_id, CAST(play_num AS int), CAST(team_id_for AS int), CAST(team_id_against AS int), event, secondaryType, CAST(x AS int), CAST(y AS int), CAST(period AS int), periodType, CAST(periodTime AS int), CAST(periodTimeRemaining AS int), CAST(dateTime AS timestamp), CAST(goals_away AS int), CAST(goals_home AS int), description, rink_side
FROM plays_raw
""")
df.printSchema()

root
 |-- play_id: string (nullable = true)
 |-- game_id: string (nullable = true)
 |-- play_num: integer (nullable = true)
 |-- team_id_for: integer (nullable = true)
 |-- team_id_against: integer (nullable = true)
 |-- event: string (nullable = true)
 |-- secondaryType: string (nullable = true)
 |-- x: integer (nullable = true)
 |-- y: integer (nullable = true)
 |-- period: integer (nullable = true)
 |-- periodType: string (nullable = true)
 |-- periodTime: integer (nullable = true)
 |-- periodTimeRemaining: integer (nullable = true)
 |-- dateTime: timestamp (nullable = true)
 |-- goals_away: integer (nullable = true)
 |-- goals_home: integer (nullable = true)
 |-- description: string (nullable = true)
 |-- rink_side: string (nullable = true)



In [74]:
df.write.format('jdbc').options(
    url='jdbc:mysql://localhost/50_in_07',
    driver='com.mysql.jdbc.Driver',
    dbtable='plays',
    user='root').mode('overwrite').save()

Plays by player information

In [5]:
player_plays_file_path = '../../data/game_plays_players.csv'
df_raw = spark.read.csv(player_plays_file_path, 
                        header=True, 
                        mode="DROPMALFORMED")
df_raw.printSchema()
df_raw.createOrReplaceTempView('player_plays_raw')
spark.sql("""
SELECT * FROM player_plays_raw LIMIT 10
""").show()

root
 |-- play_id: string (nullable = true)
 |-- game_id: string (nullable = true)
 |-- play_num: string (nullable = true)
 |-- player_id: string (nullable = true)
 |-- playerType: string (nullable = true)

+------------+----------+--------+---------+----------+
|     play_id|   game_id|play_num|player_id|playerType|
+------------+----------+--------+---------+----------+
|2011030221_4|2011030221|       4|  8476461|    Winner|
|2011030221_4|2011030221|       4|  8460542|     Loser|
|2011030221_5|2011030221|       5|  8459670|  PlayerID|
|2011030221_6|2011030221|       6|  8470171|   Blocker|
|2011030221_6|2011030221|       6|  8460542|   Shooter|
|2011030221_7|2011030221|       7|  8460542|   Shooter|
|2011030221_7|2011030221|       7|  8468524|    Goalie|
|2011030221_8|2011030221|       8|  8469623|    Hitter|
|2011030221_8|2011030221|       8|  8448208|    Hittee|
|2011030221_9|2011030221|       9|  8468486|  PlayerID|
+------------+----------+--------+---------+----------+



In [7]:
df = spark.sql("""
SELECT play_id, game_id, CAST(play_num AS int), CAST(player_id AS int), playerType
FROM player_plays_raw
""")
df.printSchema()

root
 |-- play_id: string (nullable = true)
 |-- game_id: string (nullable = true)
 |-- play_num: integer (nullable = true)
 |-- player_id: integer (nullable = true)
 |-- playerType: string (nullable = true)



In [8]:
df.write.format('jdbc').options(
    url='jdbc:mysql://localhost/50_in_07',
    driver='com.mysql.jdbc.Driver',
    dbtable='player_plays',
    user='root').mode('overwrite').save()

Shifts information

In [9]:
shifts_file_path = '../../data/game_shifts.csv'
df_raw = spark.read.csv(shifts_file_path, 
                        header=True, 
                        mode="DROPMALFORMED")
df_raw.printSchema()
df_raw.createOrReplaceTempView('shifts_raw')
spark.sql("""
SELECT * FROM shifts_raw LIMIT 10
""").show()

root
 |-- game_id: string (nullable = true)
 |-- player_id: string (nullable = true)
 |-- period: string (nullable = true)
 |-- shift_start: string (nullable = true)
 |-- shift_end: string (nullable = true)

+----------+---------+------+-----------+---------+
|   game_id|player_id|period|shift_start|shift_end|
+----------+---------+------+-----------+---------+
|2016030161|  8477964|     3|       2729|     2730|
|2016030161|  8475768|     3|       2729|     2730|
|2016030161|  8474125|     2|       1772|     1773|
|2016030161|  8470595|     4|       4212|     4214|
|2016030161|  8474618|     1|        614|      616|
|2016030161|  8470803|     2|       1819|     1821|
|2016030161|  8474716|     4|       3749|     3752|
|2016030161|  8475799|     4|       3749|     3752|
|2016030161|  8470600|     4|       3749|     3752|
|2016030161|  8470610|     4|       3749|     3752|
+----------+---------+------+-----------+---------+



In [10]:
df = spark.sql("""
SELECT game_id, CAST(player_id AS int), CAST(period AS int), CAST(shift_start AS int), CAST(shift_end AS int)
FROM shifts_raw
""")
df.printSchema()

root
 |-- game_id: string (nullable = true)
 |-- player_id: integer (nullable = true)
 |-- period: integer (nullable = true)
 |-- shift_start: integer (nullable = true)
 |-- shift_end: integer (nullable = true)



In [11]:
df.write.format('jdbc').options(
    url='jdbc:mysql://localhost/50_in_07',
    driver='com.mysql.jdbc.Driver',
    dbtable='shifts',
    user='root').mode('overwrite').save()